In [50]:
import numpy as np
import matplotlib.pyplot as plt
import sqlite3 as lite
import sys
from pyne import nucname
from pyne.material import Material

In [51]:
# get sqlite cursor
filename = '../../db/1yr.sqlite'
con = lite.connect(filename)
# allows indexing with column name
con.row_factory = lite.Row
cur = con.cursor()

# Finding the Average Assembly and its Composition 

In [52]:
# first, find the average composition and recipe 
average_burnup = cur.execute('SELECT avg(discharge_burnup) FROM discharge').fetchone()[0]
average_enrichment = cur.execute('SELECT avg(initial_enrichment) FROM discharge').fetchone()[0]
print('AVG BURNUP: %f MWD/MTHM \nAVG ENRICHMENT: %f wt%% U235' %(average_burnup, average_enrichment))

AVG BURNUP: 36169.381303 MWD/MTHM 
AVG ENRICHMENT: 3.392045 wt% U235


In [53]:
# find assembly closest to this value:
min_diff_assem_id = cur.execute('SELECT assembly_id, min(abs(discharge_burnup - %f) + abs(initial_enrichment - %f)* 10000) '
                                'FROM discharge' %(average_burnup, average_enrichment)).fetchone()[0]
print(min_diff_assem_id)

324192


In [54]:
# how different is this assembly from the average?
chosen_assem = cur.execute('SELECT * FROM discharge WHERE assembly_id = %i' %min_diff_assem_id).fetchall()
burnup = chosen_assem[0]['discharge_burnup']
enrichment = chosen_assem[0]['initial_enrichment']

# print error with the average value:
print('ERROR:')
enr_err = 100 * (average_enrichment - enrichment) / average_enrichment
bur_err = 100 * (average_burnup - burnup) / average_burnup
print('ENRICHMENT: %f %% \nBURNUP %f %%' %(enr_err, bur_err))

ERROR:
ENRICHMENT: -0.057637 % 
BURNUP -0.007240 %


In [55]:
# save this assembly's composition to a dictionary
recipe = {}
total_mass = 0
for row in chosen_assem:
    isotope = nucname.name(row['isotope'])
    mass = float(row['total_mass_g'])
    total_mass += mass
    recipe[isotope] = mass

# normalize values:
for iso in recipe:
    recipe[iso] = recipe[iso] / total_mass

print(recipe)

{'Th230': 2.2881648822788407e-09, 'Th229': 1.502397728890164e-12, 'U238': 0.9659536710780865, 'U236': 0.00446225291653653, 'Zr93': 0.00086882869395784, 'Y90': 1.399344800343854e-07, 'Zr95': 1.1033775398305739e-06, 'U233': 4.184255558015151e-09, 'U232': 1.3363535872468045e-09, 'Th232': 4.809822998684918e-09, 'U235': 0.010470484087559411, 'U234': 0.00017558666122790406, 'Pu239': 0.0075583971757774895, 'Pu240': 0.0026453112127050784, 'Pu241': 0.0017516126431537794, 'Pu242': 0.0005890043254259316, 'Ru106': 8.320559676392915e-05, 'Sb124': 7.666333242015148e-10, 'Se79': 5.187290380581019e-06, 'Sb125': 9.777386155736502e-06, 'Ru103': 6.284846250027464e-08, 'Ra226': 5.644397451435088e-14, 'Pu244': 3.979698533607694e-08, 'Sn126': 2.4386989892683627e-05, 'Sm151': 1.6099003097674687e-05, 'Tc99': 0.0008702073741377083, 'Sr90': 0.0005516566888687625, 'Am241': 0.00016437612619364937, 'Am242M': 1.2644116280326764e-06, 'C14': 1.0389150698193999e-07, 'Am243': 0.00015153847401387372, 'Cm242': 4.64974461

# We store all the assemblies in a dictionary, one with the composition with UNF-ST&DARDS, and the other with recipe composition

In [56]:
### This can be done but takes way too long to actually get each assembly
def get_assembly_dict(cur):
    """ gets assembly evalution times, isotope, total_mass_g and total mass"""
    assem_dict = {}
    query = cur.execute('SELECT * FROM discharge').fetchall()
    count = 0
    percent_done = 0
    for row in query:
        iso = nucname.name(row['isotope'])
        mass = float(row['total_mass_g'])
        if row['assembly_id'] not in assem_dict.keys():
            # first of the assembly input data, initialize assem_dict[assembly_id] dict
            assem_dict[row['assembly_id']] = {'date': row['evaluation_date']}
            assem_dict[row['assembly_id']]['mass'] = 0
            assem_dict[row['assembly_id']]['comp'] = {iso: mass}
        else:
            assem_dict[row['assembly_id']]['comp'][iso] = mass
            assem_dict[row['assembly_id']]['mass'] += mass
        count += 1
        if len(query) // 50 == count:
            percent_done += 2
            print('%s %% DONE' %percent_done)
    return assem_dict

In [64]:
def get_lump_dict(cur):
    lump_dict = {}
    query = cur.execute('SELECT evaluation_date, isotope, sum(total_mass_g) FROM discharge '
                        'GROUP BY evaluation_date, isotope').fetchall()
    print(len(query))
    for row in query:
        iso = nucname.name(row['isotope'])
        mass = float(row['sum(total_mass_g)'])
        if row['evaluation_date'] not in lump_dict.keys():
            lump_dict[row['evaluation_date']] = {}
            lump_dict[row['evaluation_date']]['mass'] = mass            
            lump_dict[row['evaluation_date']]['comp'] = {iso: mass}
        else:
            lump_dict[row['evaluation_date']]['comp'][iso] = mass
            lump_dict[row['evaluation_date']]['mass'] += mass
    print('DONE')
    return lump_dict
    

In [65]:
def convert_to_recipe(assem_dict, recipe):
    recipe_dict = assem_dict.copy()
    if sum(recipe.values()) < 0.99 or sum(recipe.values()) > 1.001:
        raise ValueError('This aint normalized son')
    for key, value in recipe_dict.items():
        recipe_dict[key]['comp'] = recipe
    return recipe_dict

In [66]:
# this takes a while:
lump_dict = get_lump_dict(cur)

110220
DONE


In [68]:
recipe_dict = convert_to_recipe(assem_dict, recipe)

# Then we convert the assembly to `pyne` material, for decay and analysis

In [ ]:
def attach_pyne_material(assem_dict):
    for key, value in assem_dict.items():
        pyne_mat = Material(value['comp'], value['mass'])
        assem_dict[key]['mat'] = pyne_mat
    return assem_dict

In [ ]:
assem_dict = attach_pyne_material(assem_dict)
recipe_dict = attach_pyne_material(recipe_dict)

# Then we decay the assemblies to 2020:

In [ ]:
def find_diff_time_secs(year, month, day, evaluation_date):
    # the UNF-ST&DARDS data format is YYYY-MM-DD
    ev_year = int(evaluation_date[:4])
    ev_month = int(evaluation_date[5:7])
    ev_day = int(evaluation_date[8:])
    
    dyear = 0
    dmonth = 0
    dday = day - ev_day
    if dday < 0:
        dmonth -= 1
        dday += 30
    dmonth += month - ev_month
    if dmonth < 0:
        dyear -= 1
        dmonth += 12
    dyear += year - ev_year
    if dyear < 0:
        raise ValueError('Cannot go back in time man')
    
    time_in_sec = dyear * (365 * 30 * 24 * 3600) + dmonth * (30 * 24 * 3600) + dday * (25 * 3600)
    return time_in_sec

In [ ]:
def decay_assemblies(assem_dict):
    decayed_dict = assem_dict.copy()
    for key, value in assem_dict.items():
        # to 2020-07-01
        decay_time = find_diff_time_secs(2020, 7, 1, key)
        assem_dict[key]['mat'] = assem_dict[key]['mat'].decay(decay_time)
    return assem_dict

In [ ]:
2020_assem_dict = decay_assemblies(assem_dict)
2020_recipe_dict = decay_assemblies(recipe_dict)

# For easier analysis, we collapse all the assemblies into one giant `pyne` material

In [ ]:
def combine_material(assem_dict):
    collapsed_mat = Material({'H1':1}, 1e-6)
    for key, value in assem_dict.items():
        decayed_material = assem_dict[key]['mat']
        collapsed_mat = collapsed_mat + decayed_material
    return collapsed_mat

# Then we compare the metrics:

In [ ]:
def decay_heat(assem_dict):
    